In [1]:
import pandas as pd
import numpy as np
import datetime
import math

from pandarallel import pandarallel
pandarallel.initialize(nb_workers=8, progress_bar=False)

INFO: Pandarallel will run on 8 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


<font color='red'>Note</font>
- Split the non-storage and storage process
    - For non-storage, use a monthly basis process
    - For storage, can process all data in one go

In future, the function for non-storage needs adjustmens:
- Not about calculation, about path and fueltype filtering

In [2]:
# Set file paths
file_path = '/Volumes/EnergyData/AER/'

bid_dayoffer_path = f'{file_path}Raw_data_zipped/BIDDAYOFFER_D'
bid_peroffer_path = f'{file_path}Raw_data_zipped/BIDPEROFFER_D'
dispatch_load_path = f'{file_path}Raw_data_zipped/DISPATCHLOAD'
dispatch_regionsum_path = f'{file_path}Raw_data_zipped/DISPATCHREGIONSUM'
dispatch_price_path = f'{file_path}Raw_data_zipped/DISPATCHPRICE'

In [3]:
# Get YearMonth to FY mapping dict
YearMonthFY = pd.read_excel(f'{file_path}Reference_data/duidinfo_SRMC.xlsx',
                         sheet_name='YearMonthFY').set_index(['Year','Month'])['FY'].to_dict()

# Data prep

## Read DUID mapping

In [4]:
def get_regional_duid_mapping(file_path,regionid):
    # Read the duid mapping table
    # It is obtained from Brian (from AER database)
    # Only SA1 DUIDs, covers both GENERATOR and LOAD
    duid_mapping = pd.read_csv(f'{file_path}Reference_data/duidinfo.csv',
                               usecols=['DUID','REGIONID','STATIONNAME','PARTICIPANTNAME',
                                        'DISPATCHTYPE','SCHEDULE_TYPE','FUELTYPE'])
    duid_mapping = duid_mapping[(duid_mapping['REGIONID']==regionid)&
                                (duid_mapping['DISPATCHTYPE']=='GENERATOR')&
                                (duid_mapping['SCHEDULE_TYPE'].isin(['SCHEDULED','SEMI-SCHEDULED']))]
    return duid_mapping

## Read DISPATCHPRICE

In [5]:
# The dispatch price (ROP and RRP) are read as reference for calculating market power.
dispatch_price = pd.DataFrame()
for year in range(2017,2024):
    for month in range(1,13):
        try:
            temp_price = pd.read_csv(f'{dispatch_price_path}/PUBLIC_DVD_DISPATCHPRICE_{str(year)}{str(month).zfill(2)}010000.zip',
                                     skiprows=1,
                                     usecols=['SETTLEMENTDATE','REGIONID','RRP','ROP',
                                              'INTERVENTION','MARKETSUSPENDEDFLAG'],
                                     parse_dates=['SETTLEMENTDATE']).dropna(subset=['REGIONID'])
            temp_price = temp_price[(temp_price['REGIONID'].isin(['NSW1','QLD1','VIC1','SA1']))&
                                    (temp_price['INTERVENTION']==0)].drop(columns=['INTERVENTION']).round(2)
            dispatch_price = pd.concat([dispatch_price,temp_price])
        except:
            pass
dispatch_price = dispatch_price.drop_duplicates(subset=['SETTLEMENTDATE','REGIONID'])
dispatch_price = dispatch_price[(dispatch_price['SETTLEMENTDATE']>=datetime.datetime(2017,7,1,0,5,0))&
                                (dispatch_price['SETTLEMENTDATE']<=datetime.datetime(2023,7,1,0,0,0))]\
                    .drop_duplicates()

# Regional collective market power

Read all regional collective data in one go as the data size is small.

In [5]:
def read_and_process_regional_collective_marketpower(regionid,dispatch_price):
    temp_srmc_bid = pd.read_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_SRMC_Supply_Curve_{regionid}.csv',
                                parse_dates=['SETTLEMENTDATE'],
                                usecols=['SETTLEMENTDATE','FY','Year','Quarter','Month','Hour',
                                         'REGIONID','RRP_simulated'])\
                        .rename(columns={'RRP_simulated':'P0'})
    temp_actual_bid = pd.read_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_Actual_Supply_Curve_{regionid}.csv',
                                  parse_dates=['SETTLEMENTDATE'],
                                  usecols=['SETTLEMENTDATE','REGIONID','RRP_simulated','RegionalTarget']).rename(columns={'RRP_simulated':'P1'})
    temp_df = temp_srmc_bid.merge(right=temp_actual_bid,
                                  on=['REGIONID','SETTLEMENTDATE'],
                                  how='left')
    temp_df = temp_df.merge(right=dispatch_price,
                            on=['REGIONID','SETTLEMENTDATE'],
                            how='left')

    temp_df = temp_df[['SETTLEMENTDATE','REGIONID',
                       'FY','Year','Quarter','Month','Hour',
                       'RegionalTarget',
                       'P0','P1','ROP','RRP',
                       'MARKETSUSPENDEDFLAG']]

    temp_df['Cost_P0'] = temp_df['RegionalTarget']*temp_df['P0']/12
    temp_df['Cost_P1'] = temp_df['RegionalTarget']*temp_df['P1']/12
    temp_df['Cost_RRP'] = temp_df['RegionalTarget']*temp_df['RRP']/12

    temp_df['MarketPower_Tentative'] = temp_df['Cost_P1']-temp_df['Cost_P0']
    temp_df['MarketPower_Effective'] = temp_df['Cost_RRP']-temp_df['Cost_P0']

    temp_df['MARKETSUSPENDEDFLAG'] = temp_df['MARKETSUSPENDEDFLAG'].fillna(0)
    temp_df = temp_df[temp_df['MARKETSUSPENDEDFLAG']==0]
    
    temp_df_agg = temp_df.groupby(by=['REGIONID','FY','Year','Quarter','Month','Hour'],
                                  as_index=False)\
                                    [['RegionalTarget','Cost_P0','Cost_P1','Cost_RRP',
                                      'MarketPower_Tentative','MarketPower_Effective']].sum()
    return temp_df_agg

## Estimating regional collective market power

In [6]:
for regionid in ['SA1']:
    print(regionid)
    temp_df_agg = read_and_process_regional_collective_marketpower(regionid,dispatch_price)
    temp_df_agg.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_collective_market_power_{regionid}.csv',
                       index=False)

SA1


In [10]:
for regionid in ['NSW1','QLD1','VIC1']:
    print(regionid)
    temp_df_agg = read_and_process_regional_collective_marketpower(regionid,dispatch_price)
    temp_df_agg.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_collective_market_power_{regionid}.csv',
                       index=False)

NSW1
QLD1
VIC1


# Regional individual market power

Read and process individual market power data at a monthly basis to avoid crashing from RAM explosion.

In [7]:
def read_and_process_individual_market_power(file_path,regionid,level,Year,Month):
    
#     # Read dispatch price data
#     temp_price = pd.read_csv(f'{dispatch_price_path}/PUBLIC_DVD_DISPATCHPRICE_{str(Year)}{str(Month).zfill(2)}010000.zip',
#                              skiprows=1,
#                              usecols=['SETTLEMENTDATE','REGIONID','RRP',
#                                       'INTERVENTION','MARKETSUSPENDEDFLAG'],
#                              parse_dates=['SETTLEMENTDATE']).dropna(subset=['REGIONID'])
#     temp_price = temp_price[(temp_price['REGIONID']==regionid)&
#                             (temp_price['INTERVENTION']==0)].drop(columns=['INTERVENTION']).round(2)
    
    # Read regional duid mapping
    duid_mapping = get_regional_duid_mapping(file_path,regionid)[['DUID','FUELTYPE','STATIONNAME','PARTICIPANTNAME']]
    
    if level == 'STATIONNAME':
        duid_mapping = duid_mapping[['STATIONNAME','PARTICIPANTNAME','FUELTYPE']]\
                            .drop_duplicates(subset=['STATIONNAME','PARTICIPANTNAME'])
    
    # Read the individual market power data, filter by the specified Year and Month
    temp_actual_bid_individual = pd.read_csv(f'{file_path}Lerner_Index/Regional_ACTUAL_Supply_Curve_{regionid}_{level}.csv',
                                         parse_dates=['SETTLEMENTDATE'])\
                                .drop(columns=['TOTALDEMAND','NETINTERCHANGE','FIXEDLOAD','DISPATCHABLELOAD'])\
                                .rename(columns={'RRP_simulated':'P1'})
    temp_actual_bid_individual = temp_actual_bid_individual[(temp_actual_bid_individual['Year']==Year)&
                                                            (temp_actual_bid_individual['Month']==Month)]    
    
    # Read the P0 data
    temp_srmc_bid = pd.read_csv(f'{file_path}Lerner_Index/Regional_SRMC_Supply_Curve_{regionid}.csv',
                            parse_dates=['SETTLEMENTDATE'],
                            usecols=['SETTLEMENTDATE','REGIONID','RRP_simulated'])\
                    .rename(columns={'RRP_simulated':'P0'})
    # Merge data
    temp_actual_bid_individual = temp_actual_bid_individual.merge(right=temp_srmc_bid,
                                                              on=['REGIONID','SETTLEMENTDATE'],
                                                              how='left')
    del temp_srmc_bid
    
    # Collect and rename column names
    individual_columns = [column.split('_')[2] for column in temp_actual_bid_individual.columns if column.startswith('RRP_simulated') ]

    for column in temp_actual_bid_individual.columns:
        if column.startswith('RRP_simulated'):
            temp_actual_bid_individual.rename(columns={column:column.split('_')[2]},inplace=True)
            
    # Convert from wide to long format
    market_power_individual = pd.melt(temp_actual_bid_individual,
                                      id_vars=['REGIONID','SETTLEMENTDATE',
                                               'RegionalTarget','P0','P1'],
                                      value_vars=individual_columns)\
                                .rename(columns={'variable':level,
                                                 'value':'P1_alt'})
    
    # Calculate the un-adjusted individual market power
    market_power_individual['MarketPower_Tentative_Ind'] = (market_power_individual['P1']-market_power_individual['P1_alt'])\
                                                                *market_power_individual['RegionalTarget']/12

    # Merge with dispatch price
    market_power_individual = market_power_individual.merge(temp_price,
                                                            on=['REGIONID','SETTLEMENTDATE'],
                                                            how='left').drop_duplicates()
    
    # Re-arrange the dataframe
    market_power_individual = market_power_individual[['REGIONID','SETTLEMENTDATE','MARKETSUSPENDEDFLAG',level,
                                                       'RegionalTarget','P0','P1_alt','P1','RRP',
                                                       'MarketPower_Tentative_Ind']]
    
    # Merge with duid mapping
    if level in ['DUID','STATIONNAME']:
        market_power_individual = market_power_individual.merge(right=duid_mapping,
                                                                on=[level],
                                                                how='left')

    # Filter data (dropping market suspended period)
    market_power_individual = market_power_individual[market_power_individual['MARKETSUSPENDEDFLAG']==0]
    
    
    # Put seasonality tags back
    market_power_individual = market_power_individual.merge(right=temp_actual_bid_individual[['SETTLEMENTDATE','FY','Year','Quarter','Month','Hour']],
                                                            on=['SETTLEMENTDATE'],
                                                            how='left')
    
    # Aggregate the result
    if level == 'STATIONNAME':
        market_power_individual_agg = market_power_individual.groupby(by=['REGIONID','PARTICIPANTNAME','STATIONNAME','FUELTYPE',
                                                                          'FY','Year','Quarter','Month','Hour'],
                                                                      as_index=False)[['MarketPower_Tentative_Ind']].sum()
    elif level == 'PARTICIPANTNAME':
        market_power_individual_agg = market_power_individual.groupby(by=['REGIONID','PARTICIPANTNAME',
                                                                          'FY','Year','Quarter','Month','Hour'],
                                                                      as_index=False)[['MarketPower_Tentative_Ind']].sum()
    
    return market_power_individual_agg

## Estimating station-level individual market power

### Non-Storage

In [8]:
level = 'STATIONNAME'
for regionid in ['SA1']:
    market_power_individual = pd.DataFrame()
    
    for Year in range(2017,2024):
        if Year == 2017:
            for Month in range(7,13):
                print(f'Processing {Year} {Month}')
                temp_market_power_individual =read_and_process_individual_market_power(file_path,dispatch_price_path,regionid,level,Year,Month)
                market_power_individual = pd.concat([market_power_individual,temp_market_power_individual])

        elif Year == 2023:
            for Month in range(1,7):
                print(f'Processing {Year} {Month}')
                temp_market_power_individual =read_and_process_individual_market_power(file_path,dispatch_price_path,regionid,level,Year,Month)
                market_power_individual = pd.concat([market_power_individual,temp_market_power_individual])
        else:
            for Month in range(1,13):
                print(f'Processing {Year} {Month}')
                temp_market_power_individual =read_and_process_individual_market_power(file_path,dispatch_price_path,regionid,level,Year,Month)
                market_power_individual = pd.concat([market_power_individual,temp_market_power_individual])
                
    market_power_individual.to_csv(f'/Volumes/EnergyData/AER/Lerner_Index/Regional_individual_market_power_{regionid}_{level}.csv',
                                   index=False)
    print(regionid,'Done.')

Processing 2017 7
Processing 2017 8
Processing 2017 9
Processing 2017 10
Processing 2017 11
Processing 2017 12
Processing 2018 1
Processing 2018 2
Processing 2018 3
Processing 2018 4
Processing 2018 5
Processing 2018 6
Processing 2018 7
Processing 2018 8
Processing 2018 9
Processing 2018 10
Processing 2018 11
Processing 2018 12
Processing 2019 1
Processing 2019 2
Processing 2019 3
Processing 2019 4
Processing 2019 5
Processing 2019 6
Processing 2019 7
Processing 2019 8
Processing 2019 9
Processing 2019 10
Processing 2019 11
Processing 2019 12
Processing 2020 1
Processing 2020 2
Processing 2020 3
Processing 2020 4
Processing 2020 5
Processing 2020 6
Processing 2020 7
Processing 2020 8
Processing 2020 9
Processing 2020 10
Processing 2020 11
Processing 2020 12
Processing 2021 1
Processing 2021 2
Processing 2021 3
Processing 2021 4
Processing 2021 5
Processing 2021 6
Processing 2021 7
Processing 2021 8
Processing 2021 9
Processing 2021 10
Processing 2021 11
Processing 2021 12
Processing 20

### Storage

In [7]:
"""
Need to add FY columns, need access to the FYYearMonth data stored in T5.
"""
def read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days):
    # Read the DUID level data
    temp_df = pd.read_csv(f'{file_path}/Lerner_Index/P1_alt/Storage/{storage_type}/Regional_ACTUAL_Supply_Curve_{regionid}_{level}_{storage_type}_{window_days}_alt.csv',
                          parse_dates=['SETTLEMENTDATE'])\
                .drop(columns=['TOTALDEMAND','NETINTERCHANGE','FIXEDLOAD','DISPATCHABLELOAD'])\
                .rename(columns={'RRP_simulated':'P1'})
    # Collect and rename column names
    individual_columns = [column.split('_')[2] for column in temp_df.columns if column.startswith('RRP_simulated') ]

    if len(individual_columns) > 0:
        for column in temp_df.columns:
            if column.startswith('RRP_simulated'):
                temp_df.rename(columns={column:column.split('_')[2]},inplace=True)

        # Convert from wide to long format
        market_power_individual = pd.melt(temp_df,
                                        id_vars=['REGIONID','SETTLEMENTDATE',
                                                'RegionalTarget','P1'],
                                        value_vars=individual_columns)\
                                    .rename(columns={'variable':level,
                                                    'value':'P1_alt'})
        market_power_individual['Year'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
        market_power_individual['Month'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
        market_power_individual['FY'] = market_power_individual.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
        market_power_individual['Quarter'] = market_power_individual['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
        market_power_individual['Hour'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)
        
        # Calculate the un-adjusted individual market power
        market_power_individual['Cost_P1'] = (market_power_individual['P1']*market_power_individual['RegionalTarget'])/12
        market_power_individual['Cost_P1_alt'] = (market_power_individual['P1_alt']*market_power_individual['RegionalTarget'])/12

        market_power_individual['MarketPower_Tentative_Ind'] = market_power_individual['Cost_P1']-market_power_individual['Cost_P1_alt']

        market_power_individual_agg = market_power_individual\
                        .groupby(by=['REGIONID',level,'FY','Year','Quarter','Month','Hour'],as_index=False)\
                        [['Cost_P1','Cost_P1_alt','MarketPower_Tentative_Ind']].sum()
        return market_power_individual_agg
    else:
        market_power_individual_agg = pd.DataFrame(columns=['REGIONID',level,'FY','Year','Quarter','Month','Hour',
                                                            'Cost_P1','Cost_P1_alt','MarketPower_Tentative_Ind'])
        return market_power_individual_agg

In [17]:
# Battery
storage_type = 'Battery'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [1,3,5]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [18]:
# Hydro_pumped
storage_type = 'Hydro_pumped'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [7,14,21,28]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [19]:
# Hydro_gas
storage_type = 'Hydro_gas'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [0]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [9]:
# Hydro_gravity
storage_type = 'Hydro_gravity'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [0]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [8]:
# Hydro_gas
storage_type = 'Hydro_gas'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [99]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [20]:
# Hydro_river
storage_type = 'Hydro_river'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [0]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}.csv',
                    index=False)

In [9]:
"""
Need to add FY columns, need access to the FYYearMonth data stored in T5.
"""
def read_and_process_individual_market_power_storage_vwadp(file_path,YearMonthFY,regionid,storage_type,level,window_days):
    # Read the DUID level data
    temp_df = pd.read_csv(f'{file_path}/Lerner_Index/P1_alt/Storage/{storage_type}/Regional_ACTUAL_Supply_Curve_{regionid}_{level}_{storage_type}_{window_days}_alt_vwadp.csv',
                          parse_dates=['SETTLEMENTDATE'])\
                .drop(columns=['TOTALDEMAND','NETINTERCHANGE','FIXEDLOAD','DISPATCHABLELOAD'])\
                .rename(columns={'RRP_simulated':'P1'})
    # Collect and rename column names
    individual_columns = [column.split('_')[2] for column in temp_df.columns if column.startswith('RRP_simulated') ]

    if len(individual_columns) > 0:
        for column in temp_df.columns:
            if column.startswith('RRP_simulated'):
                temp_df.rename(columns={column:column.split('_')[2]},inplace=True)

        # Convert from wide to long format
        market_power_individual = pd.melt(temp_df,
                                        id_vars=['REGIONID','SETTLEMENTDATE',
                                                'RegionalTarget','P1'],
                                        value_vars=individual_columns)\
                                    .rename(columns={'variable':level,
                                                    'value':'P1_alt'})
        market_power_individual['Year'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).year)
        market_power_individual['Month'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).month)
        market_power_individual['FY'] = market_power_individual.parallel_apply(lambda row: YearMonthFY[(row['Year'],row['Month'])],axis=1)
        market_power_individual['Quarter'] = market_power_individual['Month'].parallel_apply(lambda x: 'Q'+str(math.ceil(x/3)))
        market_power_individual['Hour'] = market_power_individual['SETTLEMENTDATE'].parallel_apply(lambda x: (x-datetime.timedelta(minutes=5)).hour)
        
        # Calculate the un-adjusted individual market power
        market_power_individual['Cost_P1'] = (market_power_individual['P1']*market_power_individual['RegionalTarget'])/12
        market_power_individual['Cost_P1_alt'] = (market_power_individual['P1_alt']*market_power_individual['RegionalTarget'])/12

        market_power_individual['MarketPower_Tentative_Ind'] = market_power_individual['Cost_P1']-market_power_individual['Cost_P1_alt']

        market_power_individual_agg = market_power_individual\
                        .groupby(by=['REGIONID',level,'FY','Year','Quarter','Month','Hour'],as_index=False)\
                        [['Cost_P1','Cost_P1_alt','MarketPower_Tentative_Ind']].sum()
        return market_power_individual_agg
    else:
        market_power_individual_agg = pd.DataFrame(columns=['REGIONID',level,'FY','Year','Quarter','Month','Hour',
                                                            'Cost_P1','Cost_P1_alt','MarketPower_Tentative_Ind'])
        return market_power_individual_agg

In [10]:
# Battery
storage_type = 'Battery'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [1,3,5]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage_vwadp(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}_vwadp.csv',
                    index=False)

In [12]:
# Hydro_all
storage_type = 'Hydro_all'
regionids = ['NSW1','SA1','QLD1','VIC1']
window_days_list = [7,14,21,28]
level = 'STATIONNAME'
for regionid in regionids:
    for window_days in window_days_list:
        read_and_process_individual_market_power_storage_vwadp(file_path,YearMonthFY,regionid,storage_type,level,window_days)\
            .to_csv(f'{file_path}/Lerner_Index/Aggregated_Results/Individual/Storage/{storage_type}/Regional_individual_market_power_{regionid}_{level}_{storage_type}_{window_days}_vwadp.csv',
                    index=False)